In [44]:
pip install -q pyomo

In [45]:
from pyomo.environ import * 
import pandas as pd 

In [46]:
df = pd.read_csv('Knapsack.csv',sep = ' ',header = None)

In [47]:
df.columns = ['values','weights']
df.head()

values  weights
0      26      550
1      91       84
2      72       83
3      90       43
4      46        4

In [48]:
N,max_weight = df.iloc[0]
print("N",N)
print("max_weight",max_weight)

N 26
max_weight 550


In [49]:
df.drop(0,axis = 0,inplace = True)
df.head()

values  weights
1      91       84
2      72       83
3      90       43
4      46        4
5      55       44

In [50]:
model = ConcreteModel()

In [51]:
model.x = Var(range(26),domain = Binary)

In [52]:
model.constraints = ConstraintList()
model.constraints.add(sum(model.x[i]*df['weights'].iloc[i] for i in range(26))<=max_weight)
model.objective = Objective(expr = sum(model.x[i]*df['values'].iloc[i] for i in range(26)),sense = maximize)

In [53]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    1 :    {1,}
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   26 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25}

1 Var Declarations
    x : Size=26, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True :

In [54]:
!apt-get install -y -qq coinor-cbc

In [55]:
opt_cbc = SolverFactory('cbc')

In [56]:
result = opt_cbc.solve(model)
print(result)

print("Solver Status",result.solver.status)
print("Solver termination condition",result.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: 911.0
  Upper bound: 911.0
  Number of objectives: 1
  Number of constraints: 1
  Number of variables: 26
  Number of binary variables: 26
  Number of integer variables: 26
  Number of nonzeros: 26
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.01
  Wallclock time: 0.03
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 2
  Error rc: 0
  Time: 0.07121539115905762
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver Status ok
Solver termination condition optimal


In [57]:
print("Optimal Solution:",model.objective())
subset = []
for i in range(26):
  if model.x[i].value == 0 :
    subset.append((df['values'].iloc[i],df['weights'].iloc[i]))
print(subset)

Optimal Solution: 911.0
[(35, 82), (75, 92), (15, 83), (63, 58), (29, 48), (17, 96), (40, 68), (44, 92), (40, 60), (20, 55), (12, 40)]


In [58]:
print('Solution Time',result.solver.time)

Solution Time 0.07121539115905762


In [59]:
df['ratio'] = df['values']/df['weights']
df.head()
a = df.sort_values(['ratio'],ascending = False)

In [60]:
# Greedy approach
start_time=time.time()
w=0
val=0
item=[]
K=max_weight
for i in range(26):
  w=w+a['weights'].iloc[i]
  if w>=K:
    break
  else:
    val=val+a['values'].iloc[i]
    item.append((a['weights'].iloc[i],a['values'].iloc[i]))
end_time=time.time()
time_taken_by_greedy=end_time-start_time
print('time taken to solve by Greedy method:',end_time-start_time)
print('Solution :',val)
print('subset :',item)

time taken to solve by Greedy method: 0.001814126968383789
Solution : 902
subset : [(4, 46), (14, 75), (15, 55), (25, 61), (32, 78), (18, 40), (43, 90), (12, 23), (38, 65), (56, 77), (6, 8), (44, 55), (58, 63), (84, 91), (70, 75)]


In [61]:
#solving it using dynamic programming
def knapSack(W, wt, val, n):
    K = [[0 for x in range(W + 1)] for x in range(n + 1)]

    for i in range(n + 1):
        for w in range(W + 1):
            if i == 0 or w == 0:
                K[i][w] = 0
            elif wt[i-1] <= w:
                K[i][w] = max(val[i-1]
                              + K[i-1][w-wt[i-1]],
                              K[i-1][w])
            else:
                K[i][w] = K[i-1][w]
 
    return K[n][W]

In [62]:
import time
start_time = time.time()
print(knapSack(max_weight,df['weights'].to_numpy(), df['values'].to_numpy(), N))
end_time = time.time()
print("Time taken :",end_time-start_time)

911
Time taken : 0.0345919132232666
